<a href="https://colab.research.google.com/github/DmitrijLeontev/robots_n/blob/main/DmitrijLeontev/%D0%9F%D0%BE%D0%B4%D1%85%D0%BE%D0%B4_2_%D0%91%D0%B0%D1%82%D1%87_4_%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_Whisper_Speech_Recognition_%D0%A2%D0%B8%D0%BD%D1%8C%D0%BA%D0%BE%D1%84%D1%84%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Монтируем Googl Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#установка необходимых библиотек
!pip install pydub
!pip install noisereduce
!pip install SpeechRecognition
!pip install git+https://github.com/openai/whisper.git
!pip3 install vosk
!pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-y6yw00ay
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-y6yw00ay
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Распознавание параметров аудио

In [4]:
from pydub import AudioSegment

# чтение из файла любого формата проверяем первый файл
music = AudioSegment.from_file(file='/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Голосовые_фильтры/Датасет_аудио/4/audio_2023-11-20_18-14-29.ogg',
                               format='ogg')

# продолжительность аудио (секунды)
print(music.duration_seconds)

# частота дискретизаци
print(music.frame_rate)

# количество каналов
print(music.channels)

20.5335
48000
1


### Обрезка аудио

In [5]:
# чтение из файла любого формата
Mne_v_Magadan = AudioSegment.from_file(file='/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Голосовые_фильтры/Датасет_аудио/4/audio_2023-11-20_18-14-29.ogg',
                               format='ogg')

# выведем параметры
print(Mne_v_Magadan.duration_seconds)
print(Mne_v_Magadan.frame_rate)
print(Mne_v_Magadan.channels)

# представление 40 секунд в миллисекундах
FORTY_SECONDS = 40 * 1000

# обрезка файла при помощи индексации
# возьмем первые 40 секунд аудиозаписи
Mne_v_Magadan[:FORTY_SECONDS]

20.5335
48000
1


Файл первый попался чистый без шума, на перваый взгляд.

In [6]:
# сохраним фрагмент в файл
Mne_v_Magadan[:].export('part.wav', format='wav')

<_io.BufferedRandom name='part.wav'>

In [7]:
#Переводим .ogg расширение в .wav,сделаем копию датасета  четвертого блока из 25 файлов с расширением .wav

import os
from pydub import AudioSegment

new_folder = '/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_wav'
org_folder = '/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Голосовые_фильтры/Датасет_аудио/4'

# Проверяем существует ли папка new_folder, если нет - создаем ее
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for file_name in os.listdir(org_folder):
    if file_name.endswith(".ogg"):
        file_path = os.path.join(org_folder, file_name)
        audio_file = AudioSegment.from_ogg(file_path)
        new_file_name = os.path.splitext(file_name)[0] + ".wav"
        new_file_path = os.path.join(new_folder, new_file_name)
        audio_file.export(new_file_path, format="wav")


In [8]:
# путь к папке с файлами
path = '/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_wav'

Перевел в wav. сохранил.

Транскрибируем через Wosk и Whisper и Speech recognition

In [9]:
#Основные импорты и загрузки
import pandas as pd
import noisereduce as nr
import soundfile as sf
import speech_recognition as sr
import whisper
import vosk
import wave
import json
from jiwer import wer
import librosa
import matplotlib.pyplot as plt

In [10]:
# Создадим датафрейм для работы с текущим датасетом

file_names = [i for i in os.listdir(path)]
columns = ["file_names", "length",  "text_sr", "text_whisper", "text_tinkoff", "WER_sr", "target_text",  "WER_sr", "WER_whisp" ]


#создаём пустой датафрейм, где названия файлов выступают индексами строк
final4_table = pd.DataFrame(columns=columns, index = range(0,len(file_names)))

d_bkp1 = final4_table.copy()

final4_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
final4_table['file_names'] = file_names

d_bkp2 = final4_table.copy()

final4_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,audio_2023-11-20_18-14-29.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,audio_2023-11-20_18-14-33.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audio_2023-11-20_18-14-37.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,audio_2023-11-20_18-14-40.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audio_2023-11-20_18-14-44.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,audio_2023-11-20_18-14-50.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,audio_2023-11-20_18-15-15.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,audio_2023-11-20_18-15-19.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,audio_2023-11-20_18-15-21.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,audio_2023-11-20_18-15-25.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# сборка пути для открытия файла на транскрибацию по имени

recognizer = sr.Recognizer()

#проход по каждому индексу датафрейма

for i in range(0,final4_table.shape[0]):
      # название файла из ячейки датафрейма по текущему индексу
      file_name = final4_table.at[i, 'file_names']
      # полный путь к аудиофайлу для транскрибатора
      audio_file = os.path.join(path, file_name)
      with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
      # записываем результат транскрибации в переменную
      transcription = recognizer.recognize_google(audio, language="ru-RU")
      # записываем результат в соответствующий столбец по текущему индексу
      final4_table.at[i, "text_sr"] = transcription

In [13]:
final4_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,audio_2023-11-20_18-14-29.wav,NaN,а мне нужно 20 июля поехать из улан-удэ в Мага...,NaN,NaN,NaN,NaN,NaN,NaN
1,audio_2023-11-20_18-14-33.wav,NaN,25 августа поехать из Владивостока в Анадырь О...,NaN,NaN,NaN,NaN,NaN,NaN
2,audio_2023-11-20_18-14-37.wav,NaN,28 октября мне нужно уехать в Казань из Волгог...,NaN,NaN,NaN,NaN,NaN,NaN
3,audio_2023-11-20_18-14-40.wav,NaN,поеду 10 октября из Хабаровска в Якутск обратн...,NaN,NaN,NaN,NaN,NaN,NaN
4,audio_2023-11-20_18-14-44.wav,NaN,поеду из Владивостока в Краснодар 25 августа М...,NaN,NaN,NaN,NaN,NaN,NaN
5,audio_2023-11-20_18-14-50.wav,NaN,Мне нужен на поезд желательно что-нибудь комфо...,NaN,NaN,NaN,NaN,NaN,NaN
6,audio_2023-11-20_18-15-15.wav,NaN,Мне нужно из Пятигорска 28 ноябрянин,NaN,NaN,NaN,NaN,NaN,NaN
7,audio_2023-11-20_18-15-19.wav,NaN,в феврале двадцать четвёртого года мне нужно б...,NaN,NaN,NaN,NaN,NaN,NaN
8,audio_2023-11-20_18-15-21.wav,NaN,электричка с Пятигорска до Краснодара на февра...,NaN,NaN,NaN,NaN,NaN,NaN
9,audio_2023-11-20_18-15-25.wav,NaN,4 апреля Мне нужно из Томска до ехать до Барна...,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
b_bkp3 = final4_table.copy()

In [15]:
excel_file = '/content/drive/MyDrive/Colab_Notebooks/it_on/final4_table.xlsx'

# сохраняем датафрейм в эксель. Всё читается.
final4_table.to_excel(excel_file, index=False)

In [16]:
# попробуем использовать модель виспер
# создание модели, проход по нескольким строкам с попыткой транскрибации

model_whisper = whisper.load_model("large")

for i in range(0,final4_table.shape[0]):

      file_name = final4_table.at[i, 'file_names']
      audio_file = os.path.join(path, file_name)
      result = model_whisper.transcribe(audio_file)

      final4_table.at[i, "text_whisper"] = result["text"]

100%|██████████████████████████████████████| 2.88G/2.88G [00:28<00:00, 109MiB/s]


In [17]:
final4_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,audio_2023-11-20_18-14-29.wav,NaN,а мне нужно 20 июля поехать из улан-удэ в Мага...,Мне нужно 20 июля поехать из Улан-Удэ в Магад...,NaN,NaN,NaN,NaN,NaN
1,audio_2023-11-20_18-14-33.wav,NaN,25 августа поехать из Владивостока в Анадырь О...,25 августа поехать из Владивостока в Анадырь....,NaN,NaN,NaN,NaN,NaN
2,audio_2023-11-20_18-14-37.wav,NaN,28 октября мне нужно уехать в Казань из Волгог...,28 октября мне нужно уехать в Казань из Волго...,NaN,NaN,NaN,NaN,NaN
3,audio_2023-11-20_18-14-40.wav,NaN,поеду 10 октября из Хабаровска в Якутск обратн...,"Поеду 10 октября из Хабаровска в Якутск, обра...",NaN,NaN,NaN,NaN,NaN
4,audio_2023-11-20_18-14-44.wav,NaN,поеду из Владивостока в Краснодар 25 августа М...,поеду из владивостока в краснодар 25 августа ...,NaN,NaN,NaN,NaN,NaN
5,audio_2023-11-20_18-14-50.wav,NaN,Мне нужен на поезд желательно что-нибудь комфо...,"Мне нужен поезд, желательно что-нибудь комфор...",NaN,NaN,NaN,NaN,NaN
6,audio_2023-11-20_18-15-15.wav,NaN,Мне нужно из Пятигорска 28 ноябрянин,Мне нужно из Пятигорска 28 ноября добраться д...,NaN,NaN,NaN,NaN,NaN
7,audio_2023-11-20_18-15-19.wav,NaN,в феврале двадцать четвёртого года мне нужно б...,В феврале 24 года мне нужно будет из Сочи на ...,NaN,NaN,NaN,NaN,NaN
8,audio_2023-11-20_18-15-21.wav,NaN,электричка с Пятигорска до Краснодара на февра...,Электричка с Пятигорска до Краснодара на 22 ф...,NaN,NaN,NaN,NaN,NaN
9,audio_2023-11-20_18-15-25.wav,NaN,4 апреля Мне нужно из Томска до ехать до Барна...,4 апреля мне нужно из Томска доехать до Барна...,NaN,NaN,NaN,NaN,NaN


In [18]:
b_bkp4 = final4_table.copy()

In [19]:
excel_file = '/content/drive/MyDrive/Colab_Notebooks/it_on/final4_table.xlsx'

# сохраняем датафрейм в эксель. Теперь там два варианта транскрибации
final4_table.to_excel(excel_file, index=False)

1.Whisper и Speech Recognition показали себя хорошо. Все транскрибировалось отлично. Воск не зашел.

2.Пробуем Tinkoff VoiceKit. В прошлый раз не получилось. Может формат не соответствует. Пробуем перевести в mp3 и транскрибировать.

In [10]:
#Переводим .ogg расширение в .wav,сделаем копию датасета  четвертого блока из 25 файлов с расширением .wav

import os
from pydub import AudioSegment

new_folder = '/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3'
org_folder = '/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Голосовые_фильтры/Датасет_аудио/4'

# Проверяем существует ли папка new_folder, если нет - создаем ее
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for file_name in os.listdir(org_folder):
    if file_name.endswith(".ogg"):
        file_path = os.path.join(org_folder, file_name)
        audio_file = AudioSegment.from_ogg(file_path)
        new_file_name = os.path.splitext(file_name)[0] + ".mp3"
        new_file_path = os.path.join(new_folder, new_file_name)
        audio_file.export(new_file_path, format="mp3")


In [11]:
# путь к папке с файлами
path = '/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3'

In [12]:
# чтение из файла любого формата
Mne_v_Magadan = AudioSegment.from_file(file='/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3/audio_2023-11-20_18-14-29.mp3',
                               format='mp3')

# выведем параметры
print(Mne_v_Magadan.duration_seconds)
print(Mne_v_Magadan.frame_rate)
print(Mne_v_Magadan.channels)

# представление 40 секунд в миллисекундах
FORTY_SECONDS = 40 * 1000

# обрезка файла при помощи индексации
# возьмем первые 40 секунд аудиозаписи
Mne_v_Magadan[:FORTY_SECONDS]

20.5335
48000
1


Файлы переведены в mp3 и сохранены.
Пробуем Тинькофф

# Tinkoff VoiceKit

Для использования сервиса Tinkoff VoiceKit необходима предварительная регистрация на платформе https://software.tinkoff.ru/auth/login/

После регистрации на Вашем балансе будет 1000 рублей, на которые Вы можете безвозмездно использовать сервис.

После регистрации Вам необходимо создать и сохранить 2 ключа API-key и SECRET-key. API_KEY можно сгенерировать в личном кабинете в разделе VoiceKit в любой момент времени. SECRET_KEY генерируется автоматически только при получении первого API_KEY, потом SECRET_KEY будет недоступен, поэтому крайне ВАЖНО сразу его сохранить.

In [8]:
# установка библиотек
!pip install tinkoff-voicekit-client protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.9 MB/s eta 0:00:00
     ━━

In [13]:
import getpass

# передаем API_KEY
API_KEY = getpass.getpass("Tinkoff API Key:")

Tinkoff API Key:··········


In [14]:
# передаем SECRET_KEY
SECRET_KEY = getpass.getpass("Tinkoff SECRET Key:")

Tinkoff SECRET Key:··········


## Метод **Recognize**

Используется для распознавания аудиофайлов разных форматов (mp3, wav, s16)

Для транскрибации текста, любой метод принимает словарь параметров:

    audio_config = {"encoding": "MPEG_AUDIO",  
                    "sample_rate_hertz": 16000,
                    "num_channels": 2}

    "encoding" - кодировка, может быть: 'LINEAR16', 'ALAW', 'MULAW', 'LINEAR32F', 'RAW_OPUS', 'MPEG_AUDIO';

    "sample_rate_hertz" - частота дискретизации записи;

    "num_channels" - количество каналов записи (1 или 2).


In [15]:
from tinkoff_voicekit_client import ClientSTT
from pprint import pprint

# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",
    "sample_rate_hertz": 48000,
    "num_channels": 1,
    }

# вызываем метод recognize
response = client.recognize("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3/audio_2023-11-20_18-14-29.mp3",
                            audio_config)
pprint(response)

{'results': [{'alternatives': [{'confidence': -18.68808,
                                'transcript': 'мне нужно двадцатого июля '
                                              'поехать из улан удэ в магадан',
                                'words': [{'confidence': 0.0,
                                           'end_time': '1.710s',
                                           'start_time': '1.590s',
                                           'word': 'мне'},
                                          {'confidence': 0.0,
                                           'end_time': '2.160s',
                                           'start_time': '1.920s',
                                           'word': 'нужно'},
                                          {'confidence': 0.0,
                                           'end_time': '3.210s',
                                           'start_time': '2.580s',
                                           'word': 'двадцатого'},
                     

## Метод **StreamingRecognize**

Используется для распознавания речи в реальном времени: телефонных звонков, голосовых ассистентов и т.п. Этот метод содержит много разных фильтров и опций.

In [16]:
# установка библиотеки ffmpeg-python
!pip install ffmpeg-python

In [18]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",
    "sample_rate_hertz": 48000,
    "num_channels": 1,
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3/audio_2023-11-20_18-14-29.mp3", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -18.776413,
                                                       'transcript': 'мне '
                                                                     'нужно '
                                                                     'двадцатого '
                                                                     'июля '
                                                                     'поехать '
                                                                     'из улан '
                                                                     'удэ в '
                                                                     'магадан',
                                                       'words': [{'confidence': 0.0,
                                                                  'end_time': '1.950s',
                                                                  'start_time': '1

Метод StreamingRecognize позволяет использовать дополнительные опции в словаре параметров. Вот некоторые из них:

    audio_config = {"max_alternatives": 1,  
                    "vad_config": {},
                    "enable_automatic_punctuation": False,
                    "profanity_filter": False,
                    "enable_denormalization": False}

    "max_alternatives" - количество альтернатив распознанного текста, [1,15];

    "vad_config" - настройки для обнаружения голосовой активности;

    "enable_automatic_punctuation" - пунктуация;

    "profanity_filter" - фильтр ненормативной лексики;

    "enable_denormalization" - преобразование текста в числовые данные (время, дата и тп);

    ""

### Добавляем несколько альтернативных вариантов транскрибации текста

In [19]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",   # кодировка
    "sample_rate_hertz": 48000, # частота
    "num_channels": 1,          # количество каналов
    "max_alternatives": 3       # количество альтернатив
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3/audio_2023-11-20_18-14-29.mp3", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -18.784153,
                                                       'transcript': 'мне '
                                                                     'нужно '
                                                                     'двадцатого '
                                                                     'июля '
                                                                     'поехать '
                                                                     'из улан '
                                                                     'удэ в '
                                                                     'магадан',
                                                       'words': [{'confidence': 0.0,
                                                                  'end_time': '1.950s',
                                                                  'start_time': '1

### Добавляем vad_config

In [20]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры vad (voice activity detection)
vad_config = {"silence_duration_threshold": 3,
              "silence_prob_threshold": 0.5}

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",   # кодировка
    "sample_rate_hertz": 48000, # частота
    "num_channels": 1,          # количество каналов
    "vad_config": vad_config    # vad параметры
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3/audio_2023-11-20_18-14-29.mp3", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -55.68023,
                                                       'transcript': 'мне '
                                                                     'нужно '
                                                                     'двадцатого '
                                                                     'июля '
                                                                     'поехать '
                                                                     'из улан '
                                                                     'удэ в '
                                                                     'магадан '
                                                                     'обратный '
                                                                     'путь '
                                                                     'короче '
                   

### Включаем пунктуацию

In [21]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",             # кодировка
    "sample_rate_hertz": 48000,           # частота
    "num_channels": 1,                    # количество каналов
    "enable_automatic_punctuation": True  # включаем пунктуацию
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset4_1_mp3/audio_2023-11-20_18-14-29.mp3", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -18.786,
                                                       'transcript': 'Мне '
                                                                     'нужно '
                                                                     'двадцатого '
                                                                     'июля '
                                                                     'поехать '
                                                                     'из '
                                                                     'Улан-Удэ '
                                                                     'в '
                                                                     'Магадан.',
                                                       'words': [{'confidence': 0.0,
                                                                  'end_time': '1.950s',
            

Все получилось.
Аудиофайл транскрибирован.
С пунктуацией.
Но транскрибируется не лучше , чем Whisper-ом или SpechRecognition.
И это логично. Так , как Тьньков ВойсКит создан на основе них. Но учитывая способ подачи транскрибированного текста файла и подачу по одному файлу на транскрибацию, Тинькофф проигрываает в удобстве инструмента транскрибации больших обьемов данных. По одному можно конечно поиграть , но не в нашем случае:-)

Но есть и плюсы. При распознавании Виспером и Спичрекогнишн я испоьзовал T4GPU для ускорения процесса. При распознавание Тинькоф одного файла хватило CPU.
☝